In [ ]:
# %pip install mimesis
# %pip install polars

In [1]:
import time 
import os
from mimesis import Person, Address, Numeric
import polars as pl
from concurrent.futures import ThreadPoolExecutor
import csv



In [21]:
peep = Person()

In [26]:
#peep.birthdate()
peep.gender()
peep.weight()
peep.height()

'1.94'

In [13]:

df = pl.read_csv('~/test_dummy_data/polars/data*.csv')

"""
    Update: Used the go fd fake data generator to generate 100M rows; it did it in about 5 minutes.
        it generated 14 gb of data; this is the command i used:
            ./fd create --filetype csv --maxworkers 6 --prefix fin_data_ --outputdir ~/test_dummy_data/polars --files 30 --rows 100000000

    -- add a link for that to this test harness
"""

## TODO: Aggregate data up, add timestamp column, write to parquet


10000000


In [31]:
df2 = df.group_by("zip_cd").agg(
     pl.col("net_worth").sum().name.suffix("_tot")
    ,pl.col("first_name").n_unique().name.suffix("_cnt")
    ,pl.col("city").count().alias("city_cnt")
)

df2.head(5)
df2.write_csv(file="~/test.csv",include_header=True, separator="|")

In [2]:
import polars as pl

In [ ]:
df = pl.read_csv("/Users/matthewmartin/test_dummy_data/polars/*.csv").lazy()

In [3]:
#scan the csv and put it into a lazy frame so tit doesn't gobble up all the memory
df = pl.scan_csv("/Users/matthewmartin/test_dummy_data/polars/data*.csv").lazy()

In [13]:
## aggregate up
tsf = df2.group_by('state').agg(
    pl.col("zip_cd").n_unique().alias("zip_cd_cnt"),
    pl.col("first_name").n_unique().alias("unique_first_name_cnt"),
    pl.col("net_worth").sum().alias("total_net_worth")
).collect()

tsf.head(5)

state,zip_cd_cnt,unique_first_name_cnt,total_net_worth
str,u32,u32,i64
"""Kansas""",100000,3323,99934426481
"""Alaska""",100000,3323,100052100663
"""Connecticut""",100000,3323,99981562433
"""New Mexico""",100000,3323,100023704980
"""Texas""",100000,3323,100063440157


In [11]:
import pytz
from datetime import datetime

est = pytz.timezone('America/New_York')
current_time_et = datetime.now(est)

tsf.with_columns(process_ts = pl.lit(current_time_et))

state,zip_cd_cnt,unique_first_name_cnt,process_ts
str,u32,u32,"datetime[μs, America/New_York]"
"""California""",100000,3323,2024-04-16 14:42:22.719050 EDT
"""Nebraska""",100000,3323,2024-04-16 14:42:22.719050 EDT
"""Iowa""",100000,3323,2024-04-16 14:42:22.719050 EDT
"""Arkansas""",100000,3323,2024-04-16 14:42:22.719050 EDT
"""New Mexico""",100000,3323,2024-04-16 14:42:22.719050 EDT
…,…,…,…
"""Florida""",100000,3323,2024-04-16 14:42:22.719050 EDT
"""Ohio""",100000,3323,2024-04-16 14:42:22.719050 EDT
"""Hawaii""",100000,3323,2024-04-16 14:42:22.719050 EDT


In [12]:
tsf.write_parquet("~/test_dummy_data/polars/state_agg.parquet")

In [2]:
import polars as pl

df = pl.read_parquet("~/test_dummy_data/polars/state_agg.parquet")

In [4]:
df.head(50)

state,zip_cd_cnt,unique_first_name_cnt,total_net_worth
str,u32,u32,i64
"""Ohio""",100000,3323,100058519329
"""Colorado""",100000,3323,100008262375
"""Massachusetts""",100000,3323,99979962509
"""Indiana""",100000,3323,99951159961
"""North Carolina…",100000,3323,100062421209
…,…,…,…
"""Alabama""",100000,3323,100032750413
"""Kansas""",100000,3323,99934426481
"""Wyoming""",100000,3323,99993652570
